In [18]:
!pip install groq python-dotenv

In [33]:
from groq import Groq
from getpass import getpass

# Ask user to enter API key securely (hidden input)
api_key = getpass("Enter your GROQ API key: ")

# Create client
client = Groq(api_key=api_key)

MODEL_NAME = "llama-3.3-70b-versatile"

print("✅ Groq client initialized securely")

Enter your GROQ API key: ··········
✅ Groq client initialized securely


In [34]:
MODEL_CONFIG = {
    "technical": {
        "model": MODEL_NAME,
        "system_prompt": (
            "You are a rigorous technical support engineer. "
            "Diagnose bugs, request minimal repro steps, and provide precise code fixes."
        ),
        "temperature": 0.7
    },

    "billing": {
        "model": MODEL_NAME,
        "system_prompt": (
            "You are an empathetic billing support agent. "
            "Help with refunds, subscriptions, payments and policies."
        ),
        "temperature": 0.7
    },

    "general": {
        "model": MODEL_NAME,
        "system_prompt": (
            "You are a friendly general assistant who helps with everyday questions."
        ),
        "temperature": 0.7
    }
}

In [35]:
def route_prompt(user_input):

    router_prompt = f"""
Classify the user message into ONE category:
technical
billing
general

Return ONLY the category word.

User message: {user_input}
"""

    completion = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.0,   # deterministic routing
        messages=[{"role":"user","content":router_prompt}]
    )

    category = completion.choices[0].message.content.strip().lower()
    return category

In [36]:
def process_request(user_input):

    # Step 1: Route query
    category = route_prompt(user_input)
    print("Routed to:", category)

    # Step 2: Load correct expert config
    config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    messages = [
        {"role": "system", "content": config["system_prompt"]},
        {"role": "user", "content": user_input}
    ]

    # Step 3: Call expert LLM
    completion = client.chat.completions.create(
        model=config["model"],
        temperature=config["temperature"],
        messages=messages
    )

    # Step 4: Return response
    return completion.choices[0].message.content

In [37]:
test_inputs = [
    "My Python script throws IndexError on line 5.",
    "I was charged twice for my subscription.",
    "Tell me a fun fact about space."
]

In [38]:
for text in test_inputs:
    print("\n==============================")
    print("User:", text)

    response = process_request(text)

    print("\nAssistant:", response)


User: My Python script throws IndexError on line 5.
Routed to: technical

Assistant: To better assist you, I'll need more information about the error you're encountering.

1. **Error Message**: Can you please provide the full error message you're seeing, including the `IndexError` exception?
2. **Code Snippet**: Please share the code surrounding line 5, ideally 5-10 lines before and after line 5. This will help me understand the context of the error.
3. **Input/Context**: What input or data is your script processing when the error occurs?

Once I have this information, I can provide a more accurate diagnosis and guidance on how to resolve the `IndexError`.

User: I was charged twice for my subscription.
Routed to: billing

Assistant: I'm so sorry to hear that you were charged twice for your subscription. That can be really frustrating and I'm here to help resolve the issue for you.

Can you please provide me with some more information about the duplicate charge? For example, what is t

In [39]:
def bitcoin_price_tool():
    return "Bitcoin price is approximately $60,000 (mock data)."

def route_prompt(user_input):

    router_prompt = f"""
Classify the user message into ONE category:
technical
billing
general
tool

Return ONLY the category word.

User message: {user_input}
"""

    completion = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.0,
        messages=[{"role":"user","content":router_prompt}]
    )

    return completion.choices[0].message.content.strip().lower()